# WEB SCRAPING ASSIGNMENT 3

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [82]:
#Importing reqiured libraries
import selenium
import pandas as pd
from selenium import webdriver
import requests

In [83]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime
from bs4 import BeautifulSoup

In [84]:
import warnings
warnings.filterwarnings('ignore')

In [85]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")
driver.maximize_window()

In [86]:
driver.get('http://amazon.in/')
driver.implicitly_wait(5)

In [87]:
# Let's enter the details in the search column
p_search=input("Enter product you want to search: ")
sear= driver.find_element_by_id("twotabsearchtextbox")
sear.send_keys(p_search)
sea_but=driver.find_element_by_id("nav-search-submit-button")
sea_but.click()

Enter product you want to search: laptops


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [88]:
# Let's create empty list to store data
bname=[]
pname=[]
rat=[]
nrat=[]
price=[]
ex_del=[]
ret_ex=[]
other=[]
avail=[]
p_url=[]

In [89]:
#Functions for getting products in all three pages
urls=[]
for k in range(10):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break

#Taking every URL one after one
for i in urls:
    driver.get(i)
    p_url.append(i)
    driver.implicitly_wait(5)
    
    
#Scraping the brand name
    try: bname.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: bname.append('-')
    driver.implicitly_wait(5)
    
#Scraping the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        pname.append(string)
    except: pname.append('-')
    driver.implicitly_wait(5)
    
#Scraping the rating of the product
    try: rat.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rat.append('-')
    driver.implicitly_wait(5)
#Scraping the number of ratings
    try: nrat.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: nrat.append('-')
    driver.implicitly_wait(5)
    
#Scraping the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
#Scraping the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                ret_ex.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            ret_ex.append('-')
    except: ret_ex.append('-')
    driver.implicitly_wait(5)
#Scraping the expected delivery date
    try: ex_del.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: ex_del.append('-')
    driver.implicitly_wait(5)
    
#Scraping the availablity of the product
    try: 
        try: avail.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: avail.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: avail.append('-')
    driver.implicitly_wait(5)
    
#Scraping other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other.append(details[0])
    except: other.append('-')
    driver.implicitly_wait(5)            

In [90]:
# Let's create a DataFrame for our data
amazon_lap=pd.DataFrame()

In [91]:
amazon_lap['Brand Name']=bname
amazon_lap['Name of Product']=pname
amazon_lap['Rating']=rat
amazon_lap['No. of Ratings']=nrat
amazon_lap['Price']=price
amazon_lap['Return/Exchange']=ret_ex
amazon_lap['Expected Delivery']=ex_del
amazon_lap['Availability']=avail
amazon_lap['Other details']=other
amazon_lap['Product URL']=p_url

In [92]:
amazon_lap

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [93]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")
driver.maximize_window()

In [94]:
url=('https://www.google.com/imghp?hl=en')
driver.get(url)
driver.implicitly_wait(5)

In [95]:
#Function taking image name as an argument
def get_urls(keywords):
        driver.get(url)
        
#Details in the search column and click search button
        search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_bar.send_keys(keywords)
        search_but=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        search_but.click()
        driver.implicitly_wait(5)
        
        
#loading pages to scrape images
        for k in range(12):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        picture=[image for image in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_urls=[]
        print(len(picture))
        if(len(picture)==100):
            for i in picture:
                try:
                    i.click()
                    image_urls.append(i.get_attribute('href'))
                except:
                    image_urls.append('Not Available')
            return image_urls
        else:
            print('Image Not Available')

In [96]:
#Let's scrape the url's of all images
fru=get_urls('Fruits')
car=get_urls('Cars')
m_l=get_urls('Machine Learning')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class='wXeWr islib nfEiy mM5pbd']"}
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FC06F3+2492147]
	Ordinal0 [0x00F59BD1+2071505]
	Ordinal0 [0x00E62478+1057912]
	Ordinal0 [0x00E8C964+1231204]
	Ordinal0 [0x00EB6B62+1403746]
	Ordinal0 [0x00EA57FA+1333242]
	Ordinal0 [0x00EB4F38+1396536]
	Ordinal0 [0x00EA568B+1332875]
	Ordinal0 [0x00E821D4+1188308]
	Ordinal0 [0x00E8302F+1191983]
	GetHandleVerifier [0x011467A6+1545030]
	GetHandleVerifier [0x011F105C+2243580]
	GetHandleVerifier [0x0104BC97+518199]
	GetHandleVerifier [0x0104AD80+514336]
	Ordinal0 [0x00F5ED2D+2092333]
	Ordinal0 [0x00F62EE8+2109160]
	Ordinal0 [0x00F63022+2109474]
	Ordinal0 [0x00F6CB71+2149233]
	BaseThreadInitThunk [0x75883744+36]
	RtlSetCurrentTransaction [0x7700A7B4+212]
	RtlSetCurrentTransaction [0x7700A77F+159]


In [97]:
#Creating a dataframe for storing data
img_url=pd.DataFrame({})
img_url['Fruits']=fruits
img_url['Cars']=car
img_url['Machine Learning']=m_l

NameError: name 'fruits' is not defined

img_url

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.

In [98]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [99]:
url=('https://www.flipkart.com/')
driver.get(url)
driver.implicitly_wait(5)

In [100]:
#Searching for smartphone on Flipkart
prod=input('Enter the product you want to buy : ')
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(prod))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(3)

Enter the product you want to buy : smartphones


In [101]:
#Let's create an empty list to store the scrape data
product_urls=[]
bname=[]
pname=[]
pcolor=[]
ram=[]
rom=[]
p_camera=[]
s_camera=[]
dis_size=[]
dis_res=[]
proc=[]
proc_core=[]
batt=[]
price=[]

In [102]:
#Functions for getting products links
link=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

#Let's scrape product URL
for i in link:
    product_urls.append(i.get_attribute('href'))
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        
#Let's scrape brand name
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        bname.append(title[0])
    except:
        bname.append('-')
    try:
        
#Let's scrape smartohone name
        pname.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        pname.append('-')
    try:
        
#Let's scrape smartphone colour
        pcolor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        pcolor.append('-')
    try:
        
#Let's scrape smartphone RAM
        ram.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        ram.append('-')
    try:
#Let's scrape smartphone ROM
        rom.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        rom.append('-')
    try:
        
#Let's scrape smartphone primary camera details
        p_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        p_camera.append('-')
    try:
        
#Let's scrape smartphone secondary camera details
        s_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        s_camera.append('-')
    try:
        
#Let's scrape smartphone display size
        dis_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        dis_size.append('-')
    try:
        
#Let's scrape smartphone screen resolution
        dis_res.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        dis_res.append('-')
    try:
        
#Let's scrape smartphone processor
        proc.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        proc.append('-')
    try:
#Let's scrape smartphone processor core
        proc_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        proc_core.append('-')
    try:
        
#Let's scrape smartphone battery capacity
        batt.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        batt.append('-')
    try:
        
#Let's scrape smartphone price
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')

In [103]:
#Creating a dataframe for storing the data
flip=pd.DataFrame({})
flip['Brand Name']=bname
flip['Smartphone Name']=pname
flip['Smartphone Colour']=pcolor
flip['RAM']=ram
flip['Storage (ROM)']=rom
flip['Primray Camera']=p_camera
flip['Secondary Camera']=s_camera
flip['Display Size']=dis_size
flip['Display Resolution']=dis_res
flip['Processor']=proc
flip['Processor Cores']=proc_core
flip['Battery Capacity']=batt
flip['Price']=price
flip['Product URL']=product_urls

In [104]:
flip

,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,realme,C11 2021,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,76.4 mm,"₹7,499",https://www.flipkart.com/realme-c11-2021-cool-...
1,realme,C11 2021,Cool Blue,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,76.4 mm,"₹8,999",https://www.flipkart.com/realme-c11-2021-cool-...
2,realme,C11 2021,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,76.4 mm,"₹7,499",https://www.flipkart.com/realme-c11-2021-cool-...
3,realme,C11 2021,Cool Grey,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,76.4 mm,"₹8,999",https://www.flipkart.com/realme-c11-2021-cool-...
4,realme,Narzo 50A,Oxygen Blue,4 GB,128 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,75.9 mm,"₹12,499",https://www.flipkart.com/realme-narzo-50a-oxyg...
5,realme,C25Y,Metal Grey,4 GB,128 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610 Octa Core,Octa Core,76 mm,"₹11,999",https://www.flipkart.com/realme-c25y-metal-gre...
6,APPLE,iPhone 12 Mini,Black,-,64 GB,12MP + 12MP,12MP Front Camera,13.72 cm (5.4 inch),2340 x 1080 Pixels,A14 Bionic Chip with Next Generation Neural En...,-,64.2 mm,"₹41,199",https://www.flipkart.com/apple-iphone-12-mini-...
7,REDMI,9i,Midnight Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,"₹8,499",https://www.flipkart.com/redmi-9i-midnight-bla...
8,realme,Narzo 50A,Oxygen Green,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,75.9 mm,"₹11,499",https://www.flipkart.com/realme-narzo-50a-oxyg...
9,realme,Narzo 50A,Oxygen Green,4 GB,128 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,75.9 mm,"₹12,499",https://www.flipkart.com/realme-narzo-50a-oxyg...


In [105]:
#Saving the dataframe into a csv file
flip.to_csv('flipkart-phones.csv')

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [34]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [35]:
url=('https://www.google.com/maps/')
driver.get(url)
driver.implicitly_wait(5)

In [36]:
#Function to enter the city name that you want to search 
city_name = input('Enter the name of the City you want to search : ') 
city_search=driver.find_element_by_id('searchboxinput')
city_search.send_keys(city_name)
#Function for searching the entered city
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
driver.implicitly_wait(6)

#Let's Scrape the entered city latitude and Longitude from google map
lati=driver.current_url.split('/@')[1].split(',')[0]
longi=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude of ',city_name,'is: ',lati)
print('Longitude of ',city_name,'is: ',longi)

Enter the name of the City you want to search : faridabad
Latitude of  faridabad is:  28.3277453
Longitude of  faridabad is:  77.326329


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [37]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [38]:
url=('https://trak.in/')
driver.get(url)
driver.implicitly_wait(5)

In [39]:
# Entering the details in the search column
s_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(s_button)

In [40]:
#Creating empty lists for storing the data
Date=[]
Startup_Name=[]
Industry=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [41]:
#Creating a function for getting the details
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
#Scraping the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
#Scraping the details
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup_Name.append(n.text)
    
#Scraping the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)

#Scraping the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
#Scraping the investors detils
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
#Scraping the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#Scraping the amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amount:
        Amount.append(a.text)

In [42]:
# Let's create a DataFrame for our data
trak=pd.DataFrame({})
trak['Date']=Date
trak['Startup']=Startup_Name 
trak['Industry']=Industry
trak['Location']=Location
trak['Investor']=Investor
trak['Investment']=Investment
trak['Amount']=Amount

In [43]:
trak

,Date,Startup,Industry,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


# 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [106]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [107]:
url=('https://www.digit.in/')
driver.get(url)
driver.implicitly_wait(5)

In [108]:
#Choosing top 10 from the menu
driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()

In [109]:
#Choosing top laptop in India
driver.find_element_by_xpath('//div[@class="Listbrand"]/ul/li[10]/a').click()

In [110]:
#Creating empty lists for storing the data
name = []
OS = []
display = []
processor = []
HDD = []
price = []
RAM = []
dimension = []
GPU = []
weight = []

In [111]:
#Scraping the Company Name
Name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in Name:
    name.append(i.text)
    
#Scraping the OS type
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#Scraping the display size
Display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in Display:
    display.append(i.text)
    
#Scraping the processor type
Processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in Processor:
    processor.append(i.text)
processor

#Scraping the Memory
Memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")
Memory_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") 
for i in range(len(Memory)):
        if Memory[i].text=='Memory':
            HDD.append(Memory_spec[i].text.split('/')[0])
            RAM.append(Memory_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')
            RAM.append('No details available')
#Scraping the weith
Weight=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
Weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Weight)):
        if Weight[i].text=='Weight':
            weight.append(Weight_spec[i].text)
        
#Scraping the dimensions
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#Scraping the Graphic Processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)

#Scraping the other details 
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
#Scraping a for loop to scrape the details from each page
for i in full_specs:
    driver.get(i)
    try:
        Price=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(Price.text)
    except NoSuchElementException:
        price.append("No details available")

In [112]:
# Let's create a DataFrame for our data
digit_top=pd.DataFrame({})
digit_top["Company Name: "]=Name
digit_top["OS: "]=OS
digit_top["Screen Display Size: "]=display
digit_top["HDD: "]=HDD
digit_top["RAM: "]=RAM
digit_top["Processor: "]=processor
digit_top["Weight: "]=weight
digit_top["Dimension (mm): "]=dimension
digit_top["Graphical Processor Unit: "]=GPU
digit_top["Price in Rs.: "]=price

ValueError: Length of values (0) does not match length of index (7)

In [113]:
digit_top

,Company Name:,OS:,Screen Display Size:,HDD:,RAM:
0,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10,"14"" (2160 × 1440)",256 GB SSD,8 GBGB DDR4
1,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1TB HDD,4GB DDR4
2,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"14"" (1920 X 1080)",256 GB SATA,8 GBGB DDR4
3,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME SINGLE LANGUAGE,"15.6"" (1366 X 768)",No details available,No details available
4,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SATA,8 GBGB DDR4
5,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"14"" (1920 X 1080)",512 Gb SSD,8 GBGB DDR4
6,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [114]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [115]:
url=('https://www.forbes.com/?sh=55e9467d2254')
driver.get(url)
driver.implicitly_wait(5)

In [116]:
#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billioners 
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()
time.sleep(1)

#select world billioners 
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()
time.sleep(1)

In [117]:
#make empty listes
Rank = [] 
Person_Name = [] 
total_net_worth = [] 
Age = []
country_of_citizenship = [] 
Source = []
industry = []


In [118]:
while(True):
    #scraping rank of billionaire
    rank= driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)
    time.sleep(1)
    
    #scraping name of billionaire
    name= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for i in name:
        Person_Name.append(i.text)
    time.sleep(1)
    
     #scraping Age of billionaire
    age= driver.find_elements_by_xpath("//div[@class='age']//div")
    for i in age:
        Age.append(i.text)   
    time.sleep(1)
    
    #scraping citizenship of billionaire    
    citizenship= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citizenship:
        country_of_citizenship.append(i.text)
    time.sleep(1)
    
    #scraping source of income
    source= driver.find_elements_by_xpath("//div[@class='source']")
    for i in source:
        Source.append(i.text)    
    time.sleep(1)
        
    #scraping Age of billionaire
    industries= driver.find_elements_by_xpath("//div[@class='category']//div")
    for i in industries:
        industry.append(i.text)
        
    #scraping net_worth of billionaire
    net_worth= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for i in net_worth:
        total_net_worth.append(i.text)
    time.sleep(1)    
        
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break  

        
Net_Worth = []        
for i in range(0,len(total_net_worth),2):
        Net_Worth.append(total_net_worth[i])
    

time.sleep(2)        

In [119]:
#creating dataframe
df=pd.DataFrame({'Rank':Rank,
                'Name':Person_Name,
                'Net Worth':Net_Worth,
                'Age':Age,
                'Country of Citizenship':country_of_citizenship,
                'Source':Source,
                'Industry':industry})

In [120]:
df

,Rank,Name,Net Worth,Age,Country of Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [66]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [67]:
url=('https://www.youtube.com/')
driver.get(url)
driver.implicitly_wait(5)

In [68]:
#finding element for search bar
search_bar = driver.find_element_by_id('search')
search_bar.send_keys("GOT")
time.sleep(1)

#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FC06F3+2492147]
	Ordinal0 [0x00F59BD1+2071505]
	Ordinal0 [0x00E62340+1057600]
	Ordinal0 [0x00E87FA3+1212323]
	Ordinal0 [0x00E876EF+1210095]
	Ordinal0 [0x00EA57C3+1333187]
	Ordinal0 [0x00E83466+1193062]
	Ordinal0 [0x00EA58AA+1333418]
	Ordinal0 [0x00EB4F38+1396536]
	Ordinal0 [0x00EA568B+1332875]
	Ordinal0 [0x00E821D4+1188308]
	Ordinal0 [0x00E8302F+1191983]
	GetHandleVerifier [0x011467A6+1545030]
	GetHandleVerifier [0x011F105C+2243580]
	GetHandleVerifier [0x0104BC97+518199]
	GetHandleVerifier [0x0104AD80+514336]
	Ordinal0 [0x00F5ED2D+2092333]
	Ordinal0 [0x00F62EE8+2109160]
	Ordinal0 [0x00F63022+2109474]
	Ordinal0 [0x00F6CB71+2149233]
	BaseThreadInitThunk [0x75883744+36]
	RtlSetCurrentTransaction [0x7700A7B4+212]
	RtlSetCurrentTransaction [0x7700A77F+159]


In [69]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [70]:
#make empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

In [71]:
#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
time.sleep(5)


# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
    
# scrape the comment likes
like = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
    
    
time.sleep(2)


In [72]:
#creating dataframe
df=pd.DataFrame({'Comments':comments,
                'Time':comment_time,
                'Likes':No_of_Likes})

In [73]:
df

,Comments,Time,Likes


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [74]:
#Connecting to web driver
driver = webdriver.Chrome(r"C:\Users\Administrator\Downloads\chromedriver_win32\chromedriver.exe")

In [75]:
url=('https://www.hostelworld.com/')
driver.get(url)
driver.implicitly_wait(5)

In [76]:
#locating the location search bar
search_loc = driver.find_element_by_id('search-input-field')
# write Lonodn in search bar
search_loc.send_keys("London")
time.sleep(2)

#select london
london = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(2)


# do click in search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search-input-field"]"}
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FC06F3+2492147]
	Ordinal0 [0x00F59BD1+2071505]
	Ordinal0 [0x00E62478+1057912]
	Ordinal0 [0x00E8C964+1231204]
	Ordinal0 [0x00EB6B62+1403746]
	Ordinal0 [0x00EA57FA+1333242]
	Ordinal0 [0x00EB4F38+1396536]
	Ordinal0 [0x00EA568B+1332875]
	Ordinal0 [0x00E821D4+1188308]
	Ordinal0 [0x00E8302F+1191983]
	GetHandleVerifier [0x011467A6+1545030]
	GetHandleVerifier [0x011F105C+2243580]
	GetHandleVerifier [0x0104BC97+518199]
	GetHandleVerifier [0x0104AD80+514336]
	Ordinal0 [0x00F5ED2D+2092333]
	Ordinal0 [0x00F62EE8+2109160]
	Ordinal0 [0x00F63022+2109474]
	Ordinal0 [0x00F6CB71+2149233]
	BaseThreadInitThunk [0x75883744+36]
	RtlSetCurrentTransaction [0x7700A7B4+212]
	RtlSetCurrentTransaction [0x7700A77F+159]


In [77]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [78]:
while(True):
    
    # Hostel name
    names = driver.find_elements_by_xpath('//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements_by_xpath('//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements_by_xpath('//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements_by_xpath('//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements_by_xpath('//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)        
        
# Separate  Privates_From price  and  Dorms_From price
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])
time.sleep(2)

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])           

In [79]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break


        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [80]:
#creating dataframe
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[3:],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':Privates_From,
                'Dorms From Price':Dorms_From,
                'Rating':Rating,
                'Property Description':property_description})

In [81]:
df

,Hostel Name,Distance from city centre,Overall Review,Total Reviews,Facilities,Privates From Price,Dorms From Price,Rating,Property Description
